In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [1]:
allam_model = AutoModelForCausalLM.from_pretrained("model", torch_dtype=torch.float16, device_map="auto")
messages=[
    {"role": "user", "content": "كيف أجهز كوب شاهي؟"},
]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
tokenizer = AutoTokenizer.from_pretrained("model")

~15GB Ram

In [3]:
inputs = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
inputs = {k: v.to('cuda') for k,v in inputs.items()}
# allam_model = allam_model.to('cuda')
response = allam_model.generate(**inputs, max_new_tokens=4096, do_sample=True, top_k=50, top_p=0.95, temperature=.6)
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] كيف أجهز كوب شاهي؟ [/INST] لإعداد كوب من الشاي اللذيذ، يمكنك اتباع الخطوات التالية:

المكونات:
1. شاي أسود (حسب الرغبة والكمية المطلوبة)
2. ماء (حسب الحاجة)
3. سكر أو عسل (حسب الذوق)
4. حليب (اختياري)
5. أوراق النعناع (اختياري)

الخطوات:
1. املأ غلاية الشاي بالماء الطازج والغليان واتركها حتى تغلي.
2. ضع كمية الشاي الأسود المطلوبة في مصفاة الشاي أو كيس شاي.
3. إذا كنت تستخدم مصفاة الشاي، ضعها داخل الكوب الذي تريد تقديم الشاي فيه.
4. إذا كنت تستخدم كيس شاي، ضعه مباشرة في الكوب.
5. صب الماء المغلي فوق الشاي واتركه ينقع لمدة 2-5 دقائق (حسب الرغبة في قوة الشاي).
6. بعد انتهاء النقع، إذا رغبت في إضافة الحليب، ضعه في كوب منفصل وأضف السكر أو العسل حسب الذوق.
7. قلب الحليب والسكر حتى يذوب السكر تمامًا.
8. صب الحليب المحلى فوق الشاي واتركه يمتزج جيدًا.
9. إذا أردت نكهة النعناع، أضف أوراق النعناع الطازجة إلى الكوب وقلبها مع الشاي قبل إضافة الحليب والسكر.
10. يمكنك تعديل كمية الشاي والماء والحليب والسكر حسب تفضيلك الشخصي.
11. استمتع بكوب الشاي المحضر حديثًا.

تذكر أن تعدل مدة النقع وكمية ال

## Tokenize

In [4]:
print(tokenizer.all_special_tokens)

['<s>', '</s>', '<unk>']


In [6]:
sentence1 = "ترى راسي ما عاد يتحمل الغازك"
sentence2 = "واخرتها يا زيد اتموتني وانا حي"
sentence3 = "هنا يا واد انا جاي مشوار قوم يلا"
for sentence in [sentence1, sentence2, sentence3]:
    tokens = tokenizer.tokenize(sentence)
    print(tokens)

['▁ترى', '▁راسي', '▁ما', '▁عاد', '▁يتحمل', '▁الغاز', 'ك']
['▁واخر', 'تها', '▁يا', '▁زيد', '▁اتم', 'وت', 'ني', '▁وانا', '▁حي']
['▁هنا', '▁يا', '▁واد', '▁انا', '▁جاي', '▁مشوار', '▁قوم', '▁يلا']


In [11]:
import re
sentence_with_tashkeel = "إِنَّ المَعرِفَةَ قُوَّةٌ، وَالعِلمُ نُورٌ"
tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
text_after_removing_tashkeel = re.sub(tashkeel, '', sentence_with_tashkeel)
print(text_after_removing_tashkeel)

إن المعرفة قوة، والعلم نور


# Adding Special Tokens
- `<|Najdi|>` and `</Najdi|>` : Najdi Arabic
- `<|Hijazi|>` and `</Hijazi|>` : Hijazi Arabic
- `<|Khaliji|>` and `</Khaliji|>` : Khaliji Arabic
- `<|MSA|>` and `</MSA|>` : Modern Standard Arabic

In [2]:
allam_tokenizer = AutoTokenizer.from_pretrained("model")

In [3]:
len(allam_tokenizer)

64000

In [4]:
tokens = allam_tokenizer.tokenize("Speaker2: <|Najdi|> سلامة راسك يا ابو مسامح </Najdi|>")
print(tokens)

['▁Speaker', '2', ':', '▁<', '|', 'N', 'aj', 'di', '|', '>', '▁سلامة', '▁راسك', '▁يا', '▁ابو', '▁مس', 'امح', '▁</', 'N', 'aj', 'di', '|', '>']


In [5]:
added_tokens = allam_tokenizer.add_special_tokens({'additional_special_tokens': ['<|Najdi|>', '</Najdi|>', '<|Hijazi|>', '</Hijazi|>', '<|Khaliji|>', '</Khaliji|>', '<|MSA|>', '</MSA|>']})
print(added_tokens)

8


In [6]:
allam_model = AutoModelForCausalLM.from_pretrained("model", torch_dtype=torch.float16, device_map="auto")

allam_model.resize_token_embeddings(len(allam_tokenizer))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Embedding(64008, 4096)

In [7]:
len(allam_tokenizer)

64008

In [8]:
allam_tokenizer.save_pretrained('allam_v0')

allam_model.save_pretrained('allam_v0')

In [2]:
allam_model = AutoModelForCausalLM.from_pretrained("allam_v0", torch_dtype=torch.float16, device_map="auto")
allam_tokenizer = AutoTokenizer.from_pretrained("allam_v0")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
tokens = allam_tokenizer.tokenize("Speaker2: <|Najdi|> سلامة راسك يا ابو مسامح </Najdi|>")
print(tokens)

['▁Speaker', '2', ':', '▁', '<|Najdi|>', '▁سلامة', '▁راسك', '▁يا', '▁ابو', '▁مس', 'امح', '▁', '</Najdi|>']


In [6]:
assert allam_model.get_input_embeddings().weight.shape[0] == len(allam_tokenizer)